In [24]:
import pandas as pd
import numpy as np
import datetime as dt
import altair as alt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Read the CSV file
data = pd.read_csv('Resources/dataset.csv')

# Display the dataprint(data)
data.head(10)

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,6.705989e+07,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
5,6,104000,Technology,Bill Gates,67.0,United States,Medina,Microsoft,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
6,7,94500,Media & Entertainment,Michael Bloomberg,81.0,United States,New York,Bloomberg LP,Media & Entertainment,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
7,8,93000,Telecom,Carlos Slim Helu & family,83.0,Mexico,Mexico City,Telecom,Telecom,Mexico,...,3.6,"$1,258,286,717,125",40.2,105.8,75.0,13.1,55.1,1.260140e+08,23.634501,-102.552784
8,9,83400,Diversified,Mukesh Ambani,65.0,India,Mumbai,Diversified,Diversified,India,...,7.7,"$2,611,000,000,000",28.1,113.0,69.4,11.2,49.7,1.366418e+09,20.593684,78.962880
9,10,80700,Technology,Steve Ballmer,67.0,United States,Hunts Point,Microsoft,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891


In [25]:
# Check for null values
missing_values = data.isnull().sum()
print(missing_values)

rank                                             0
finalWorth                                       0
category                                         0
personName                                       0
age                                             65
country                                         38
city                                            72
source                                           0
industries                                       0
countryOfCitizenship                             0
organization                                  2315
selfMade                                         0
status                                           0
gender                                           0
birthDate                                       76
lastName                                         0
firstName                                        3
title                                         2301
date                                             0
state                          

In [26]:
# Drop unneeded columns
data_dropped = data.drop(columns=['birthMonth', 'birthDay', 'cpi_change_country', 'gross_tertiary_education_enrollment', 'gross_primary_education_enrollment_country', 'life_expectancy_country', 'tax_revenue_country_country', 'total_tax_rate_country', 'organization', 'title', 'residenceStateRegion', 'date', 'state', 'birthYear', 'status'])
data_dropped.head(10)

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,selfMade,gender,birthDate,lastName,firstName,cpi_country,gdp_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,False,M,3/5/1949 0:00,Arnault,Bernard,110.05,"$2,715,518,274,227",6.705989e+07,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,True,M,6/28/1971 0:00,Musk,Elon,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,True,M,1/12/1964 0:00,Bezos,Jeff,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,True,M,8/17/1944 0:00,Ellison,Larry,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,True,M,8/30/1930 0:00,Buffett,Warren,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
5,6,104000,Technology,Bill Gates,67.0,United States,Medina,Microsoft,Technology,United States,True,M,10/28/1955 0:00,Gates,Bill,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
6,7,94500,Media & Entertainment,Michael Bloomberg,81.0,United States,New York,Bloomberg LP,Media & Entertainment,United States,True,M,2/14/1942 0:00,Bloomberg,Michael,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
7,8,93000,Telecom,Carlos Slim Helu & family,83.0,Mexico,Mexico City,Telecom,Telecom,Mexico,True,M,1/28/1940 0:00,Slim Helu,Carlos,141.54,"$1,258,286,717,125",1.260140e+08,23.634501,-102.552784
8,9,83400,Diversified,Mukesh Ambani,65.0,India,Mumbai,Diversified,Diversified,India,False,M,4/19/1957 0:00,Ambani,Mukesh,180.44,"$2,611,000,000,000",1.366418e+09,20.593684,78.962880
9,10,80700,Technology,Steve Ballmer,67.0,United States,Hunts Point,Microsoft,Technology,United States,True,M,3/24/1956 0:00,Ballmer,Steve,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891


In [27]:
# The following commented-out line has been run once in order to put the column 'finalWorth' in dollars
# instead of millions of dollars.

#data_dropped['finalWorth'] = data_dropped['finalWorth']/1000

data_dropped.rename(columns = {'latitude_country': 'country_lat', 'longitude_country': 'country_long', 'cpi_country': 'country_cpi', 'gdp_country': 'country_gdp', 'population_country': 'country_population'}, inplace = True)
data_dropped.head(10)

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,selfMade,gender,birthDate,lastName,firstName,country_cpi,country_gdp,country_population,country_lat,country_long
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,False,M,3/5/1949 0:00,Arnault,Bernard,110.05,"$2,715,518,274,227",6.705989e+07,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,True,M,6/28/1971 0:00,Musk,Elon,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,True,M,1/12/1964 0:00,Bezos,Jeff,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,True,M,8/17/1944 0:00,Ellison,Larry,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,True,M,8/30/1930 0:00,Buffett,Warren,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
5,6,104000,Technology,Bill Gates,67.0,United States,Medina,Microsoft,Technology,United States,True,M,10/28/1955 0:00,Gates,Bill,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
6,7,94500,Media & Entertainment,Michael Bloomberg,81.0,United States,New York,Bloomberg LP,Media & Entertainment,United States,True,M,2/14/1942 0:00,Bloomberg,Michael,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891
7,8,93000,Telecom,Carlos Slim Helu & family,83.0,Mexico,Mexico City,Telecom,Telecom,Mexico,True,M,1/28/1940 0:00,Slim Helu,Carlos,141.54,"$1,258,286,717,125",1.260140e+08,23.634501,-102.552784
8,9,83400,Diversified,Mukesh Ambani,65.0,India,Mumbai,Diversified,Diversified,India,False,M,4/19/1957 0:00,Ambani,Mukesh,180.44,"$2,611,000,000,000",1.366418e+09,20.593684,78.962880
9,10,80700,Technology,Steve Ballmer,67.0,United States,Hunts Point,Microsoft,Technology,United States,True,M,3/24/1956 0:00,Ballmer,Steve,117.24,"$21,427,700,000,000",3.282395e+08,37.090240,-95.712891


In [28]:
data_dropped.columns

Index(['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship', 'selfMade',
       'gender', 'birthDate', 'lastName', 'firstName', 'country_cpi',
       'country_gdp', 'country_population', 'country_lat', 'country_long'],
      dtype='object')

In [29]:
# Import 'worldcities.csv' file to merge on city column. This will give us coordinates for each city.
#world_cities_raw = pd.read_csv('Resources/worldcities.csv')

# Drop unnecessary columns
#world_cities = world_cities_raw.drop(columns = ['city', 'iso2', 'iso3', 'admin_name', 'capital', 'population', 'id'])

# Rename columns for consistency
#world_cities.rename(columns = {'city_ascii': 'city', 'lat': 'city_lat', 'lng': 'city_long'}, inplace = True)
#world_cities.head(10)

In [30]:
#data_clean = pd.merge(data_dropped, world_cities, on = ['city','country'], how = 'left')
#data_clean.head(10)

In [31]:
# Specify the filename
output_filename = 'Resources/data_clean.csv'

# Save the DataFrame to a CSV file
data_dropped.to_csv(output_filename, index=False)


**Altair Visualizations: Scatter Matrix**

In [32]:
# Connect cleaned data csv as the source
from vega_datasets import data
source = 'Resources/data_clean.csv'


In [33]:
# Make scatter matrix with age, finalWorth, gdp, cpi, and population
# Used code provided in Altair's documentation: https://altair-viz.github.io/gallery/scatter_matrix.html
alt.Chart(source).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Origin:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['age', 'finalWorth', 'country_cpi', 'country_gdp', 'country_population'],
    column=['country_population', 'country_gdp', 'country_cpi', 'finalWorth', 'age']
).interactive()

alt.RepeatChart(...)

In [34]:
# Make scatter matrix with age. finalWorth, and country_population
alt.Chart(source).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Origin:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['age', 'finalWorth', 'country_population'],
    column=['country_population', 'finalWorth', 'age']
).interactive()

alt.RepeatChart(...)

**Altair Visualizations: Boxplot with Min/Max Whiskers**

In [35]:
# Used code from Altair's documentation: https://altair-viz.github.io/gallery/boxplot.html
# Age for X-axis and finalWorth for y-axis
alt.Chart(source).mark_boxplot(extent='min-max').encode(
    x='age:O',
    y='finalWorth:Q'
)

alt.Chart(...)

In [36]:
# Population for X-axis and age for y-axis
alt.Chart(source).mark_boxplot(extent='min-max').encode(
    x='country_population:O',
    y='age:Q'
)

alt.Chart(...)

In [37]:
# Population for X-axis and finalWorth for y-axis
alt.Chart(source).mark_boxplot(extent='min-max').encode(
    x='country_population:O',
    y='finalWorth:Q'
)

alt.Chart(...)

**Altair Visualizations: Stacked Bar Chart with Sorted Segments**

In [42]:
# Used code from Altair's documentation: https://altair-viz.github.io/gallery/stacked_bar_chart_sorted_segments.html
# Used Xpert Learning Assitant to refine code with data types in order to return stacked bar chart
STACKEDbarCHART_finalWorth_countries_industries = alt.Chart(source).mark_bar().encode(
    x='sum(finalWorth):Q',  # Ensure the type is quantitative
    y=alt.Y('countryOfCitizenship:N', sort='-x'),  # Sort by x-axis values
    color='industries:N',  # Ensure industries is treated as a nominal variable
    order=alt.Order(
        'industries:N',  # Specify that industries is nominal
        sort='ascending'
    )
).properties(
    title='Stacked Bar Chart of Final Worth by Country and Industries'
)
STACKEDbarCHART_finalWorth_countries_industries

alt.Chart(...)

In [52]:
# Stacked bar chart of finalWorth by Industries and Countries
# Used Xpert Learning Assitant to refine code with data types in order to return stacked bar chart
StackedBarChart_FW_Ind_Countries = alt.Chart(source).mark_bar().encode(
    x='sum(finalWorth):Q',  # Ensure the type is quantitative
    y=alt.Y('industries:N', sort='-x'),  # Sort by x-axis values
    color='countryOfCitizenship:N',  # Ensure countryOfCitizenship is treated as a nominal variable
    order=alt.Order(
        'countryOfCitizenship:N',  # Specify that countryOfCitizenship is nominal
        sort='descending'
    )
).properties(
    title='Stacked Bar Chart of Final Worth by Country and Industries'
)

StackedBarChart_FW_Ind_Countries

alt.Chart(...)

**Altair Visualizations: Normalized Stacked Bar Chart**

In [53]:
# Used code from Altair's documentation: https://altair-viz.github.io/gallery/normalized_stacked_bar_chart.html
# Used Xpert Learning Assitant to refine code with data types in order to return normalized, stacked bar chart
NORMALIZEDbarCHART_FW_CoC_Ind = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(finalWorth):Q', stack='normalize'),  # Ensure 'finalWorth' is treated as quantitative
    y=alt.Y('countryOfCitizenship:N'),  # Ensure 'countryOfCitizenship' is treated as nominal
    color='industries:N'  # Ensure 'industries' is treated as nominal
)
NORMALIZEDbarCHART_FW_CoC_Ind

alt.Chart(...)

In [55]:
NORMALIZEDbarCHART_FW_CoC_Gender = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(finalWorth):Q', stack='normalize'),  # Ensure 'finalWorth' is treated as quantitative
    y=alt.Y('countryOfCitizenship:N'),  # Ensure 'countryOfCitizenship' is treated as nominal
    color='gender:N'  # Ensure 'gender' is treated as nominal
)
NORMALIZEDbarCHART_FW_CoC_Gender

alt.Chart(...)

In [56]:
NORMALIZEDbarCHART_FW_CoC_SelfMade = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(finalWorth):Q', stack='normalize'),  # Ensure 'finalWorth' is treated as quantitative
    y=alt.Y('countryOfCitizenship:N'),  # Ensure 'countryOfCitizenship' is treated as nominal
    color='selfMade:N'  # Ensure 'selfMade' is treated as nominal
)
NORMALIZEDbarCHART_FW_CoC_SelfMade

alt.Chart(...)

In [58]:
NORMALIZEDbarCHART_FW_Ind_selfMade = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(finalWorth):Q', stack='normalize'),  # Ensure 'finalWorth' is treated as quantitative
    y=alt.Y('industries:N'),  # Ensure 'countryOfCitizenship' is treated as nominal
    color='selfMade:N'  # Ensure 'selfMade' is treated as nominal
)
NORMALIZEDbarCHART_FW_Ind_selfMade

alt.Chart(...)

In [59]:
NORMALIZEDbarCHART_FW_Ind_Gender = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(finalWorth):Q', stack='normalize'),  # Ensure 'finalWorth' is treated as quantitative
    y=alt.Y('industries:N'),  # Ensure 'countryOfCitizenship' is treated as nominal
    color='gender:N'  # Ensure 'gender' is treated as nominal
)
NORMALIZEDbarCHART_FW_Ind_Gender

alt.Chart(...)

**Altair Visualizations: Selection Histogram**

In [60]:
# Used code from Altair's documentation: https://altair-viz.github.io/gallery/selection_histogram.html
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='age:Q',
    color=alt.condition(brush, 'countryOfCitizenship:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='countryOfCitizenship:N',
    color='countryOfCitizenship:N',
    x='count(countryOfCitizenship):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [61]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='age:Q',
    color=alt.condition(brush, 'industries:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='industries:N',
    color='industries:N',
    x='count(industries):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [62]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='age:Q',
    color=alt.condition(brush, 'selfMade:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='selfMade:N',
    color='selfMade:N',
    x='count(selfMade):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [66]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='age:Q',
    color=alt.condition(brush, 'gender:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='gender:N',
    color='gender:N',
    x='count(gender):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [63]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='country_population:Q',
    color=alt.condition(brush, 'countryOfCitizenship:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='countryOfCitizenship:N',
    color='countryOfCitizenship:N',
    x='count(countryOfCitizenship):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [64]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='country_population:Q',
    color=alt.condition(brush, 'industries:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='industries:N',
    color='industries:N',
    x='count(industries):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [65]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='country_population:Q',
    color=alt.condition(brush, 'selfMade:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='selfMade:N',
    color='selfMade:N',
    x='count(selfMade):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [67]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='finalWorth:Q',
    y='country_population:Q',
    color=alt.condition(brush, 'gender:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='gender:N',
    color='gender:N',
    x='count(gender):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [68]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='country_population:Q',
    y='age:Q',
    color=alt.condition(brush, 'countryOfCitizenship:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='countryOfCitizenship:N',
    color='countryOfCitizenship:N',
    x='count(countryOfCitizenship):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [69]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='country_population:Q',
    y='age:Q',
    color=alt.condition(brush, 'industries:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='industries:N',
    color='industries:N',
    x='count(industries):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [70]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='age:Q',
    y='country_population:Q',
    color=alt.condition(brush, 'selfMade:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='selfMade:N',
    color='selfMade:N',
    x='count(selfMade):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

In [71]:
brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='age:Q',
    y='country_population:Q',
    color=alt.condition(brush, 'gender:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='gender:N',
    color='gender:N',
    x='count(gender):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)